In [1]:
from torch.utils.data import Dataset,DataLoader
import torch.nn as nn
from torch.nn import Module
from torch.nn import Conv2d
from torch.nn import Linear
from torch.nn import MaxPool2d
from torch.nn import ReLU
from torch.nn import LogSoftmax
from torch.nn import BatchNorm2d
from torch import sigmoid
from torch import flatten
from PIL import Image
import numpy as np
import pandas as pd
import torch
from torchvision import transforms

In [2]:
import os
from google.colab import drive
drive.mount('/content/drive')
os.chdir('/content/drive/My Drive/FYP_CNN')
os.getcwd()

Mounted at /content/drive


'/content/drive/My Drive/FYP_CNN'

In [3]:
class CNN(nn.Module):
  def __init__(self,batchSize):
      super(CNN,self).__init__()
      #input: 1*64*64 greyScale so 1 channel
      self.layer1=nn.Sequential(
          # Defining a 2D convolution layer
          Conv2d(in_channels=1, out_channels=32, kernel_size=3, stride=1, padding=1),
          ReLU(inplace=True),
          MaxPool2d(kernel_size=2, stride=2)
      )
      #32*32*32
      self.layer2=nn.Sequential(
          # Defining a 2D convolution layer
          Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1),
          
          ReLU(inplace=True),
          MaxPool2d(kernel_size=2, stride=2)
      )
      #64*16*16
      self.layer3=nn.Sequential(
          # Defining a 2D convolution layer
          Conv2d(in_channels=64, out_channels=64, kernel_size=3, stride=1, padding=1),
          ReLU(inplace=True),
          MaxPool2d(kernel_size=2, stride=2)
      )
      #64*8*8
      self.fc1 = nn.Linear(in_features=64*8*8,out_features=200)
      self.fc2 = nn.Linear(in_features=200,out_features=1)
      # -inf, inf
        
  def forward(self,x):
      
      x=self.layer1(x)
      
      x=self.layer2(x)
      
      x=self.layer3(x)
      
      x=self.fc1(flatten(x,1))
      x=self.fc2(x)
      result=sigmoid(x)
      return result

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
fypCNN = CNN(64)
model_dict =torch.load("/content/drive/My Drive/FYP_CNN/tensors.pt",map_location=torch.device('cpu'))
fypCNN.load_state_dict(model_dict)
fypCNN = fypCNN.to(device)

In [5]:
img=Image.open("/content/drive/My Drive/FYP_CNN/test2/undistorted%d.png"%(1))  # replace the image location 
trans_resize=transforms.Resize([64,64])
trans_tograyscale=transforms.Grayscale(num_output_channels=1)
trans_totensor=transforms.ToTensor()
trans_compose=transforms.Compose([trans_resize,trans_tograyscale,trans_totensor])
img_out=trans_compose(img).to(device)
img_mean=img_out.mean()
std=0.225 
img_out=(img_out-img_mean)/std
img_out=img_out.unsqueeze(0)
print(fypCNN(img_out))

tensor([[0.9956]], grad_fn=<SigmoidBackward0>)
tensor([[1.0000]], grad_fn=<SigmoidBackward0>)
tensor([[0.9996]], grad_fn=<SigmoidBackward0>)
tensor([[1.0000]], grad_fn=<SigmoidBackward0>)
tensor([[0.9926]], grad_fn=<SigmoidBackward0>)
tensor([[0.3146]], grad_fn=<SigmoidBackward0>)
tensor([[1.0000]], grad_fn=<SigmoidBackward0>)
tensor([[0.1791]], grad_fn=<SigmoidBackward0>)
tensor([[0.9925]], grad_fn=<SigmoidBackward0>)
tensor([[0.9877]], grad_fn=<SigmoidBackward0>)
